In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [2]:
train_data, validation_data, test = tfds.load(
    name='imdb_reviews', 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)


In [3]:
train_data


<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [4]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))


In [5]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [7]:
embed = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embed, input_shape = [], dtype = tf.string,trainable = True)

In [8]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [11]:
class MyHubLayer(tf.keras.layers.Layer): 
    def __init__(self, hub_layer): 
        super(MyHubLayer, self).__init__() 
        self.hub_layer = hub_layer 
    def call(self, inputs): 
        return self.hub_layer(inputs)

In [12]:
inputs = tf.keras.Input(shape=[], dtype=tf.string) 
x = MyHubLayer(hub_layer)(inputs) 
x = tf.keras.layers.Dense(16, activation='relu')(x) 
outputs = tf.keras.layers.Dense(1)(x) 
model = tf.keras.Model(inputs, outputs)

In [13]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ my_hub_layer (MyHubLayer)       │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 353 (1.38 KB)

 Trainable params: 353 (1.38 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [15]:
history = model.fit(
    train_data.shuffle(10000).batch(100),
    epochs=25,
    validation_data=validation_data.batch(100),
    verbose=1
)

Epoch 1/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5463 - loss: 1.0093 - val_accuracy: 0.6209 - val_loss: 0.6294
Epoch 2/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6329 - loss: 0.6207 - val_accuracy: 0.6534 - val_loss: 0.6040
Epoch 3/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6500 - loss: 0.6011 - val_accuracy: 0.6662 - val_loss: 0.5934
Epoch 4/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6593 - loss: 0.5934 - val_accuracy: 0.6691 - val_loss: 0.5865
Epoch 5/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6742 - loss: 0.5804 - val_accuracy: 0.6726 - val_loss: 0.5818
Epoch 6/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6710 - loss: 0.5833 - val_accuracy: 0.6666 - val_loss: 0.5783
Epoch 7/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6667 - loss: 0.5811 - val_accuracy: 0.6777 - val_loss: 0.5726
Epoch 8/25
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6735 - loss: 0.5783 - val_accuracy: 0.

In [16]:
results = model.evaluate(test.batch(100), verbose = 2)
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f"%(name, value))

250/250 - 1s - 5ms/step - accuracy: 0.7040 - loss: 0.5494
loss: 0.549
compile_metrics: 0.704
